In [149]:
import numpy as np 
import pandas as pd 
import requests as re
import json
import urllib.request
import bs4 as bs

In [150]:
pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [168]:
from tmdbv3api import TMDb

In [169]:
url = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [170]:
source = urllib.request.urlopen(url).read()
scrap = bs.BeautifulSoup(source,'lxml')

In [172]:
tables = scrap.find_all('table', class_='wikitable sortable')

In [173]:
len(tables)

4

In [174]:
content1 = pd.read_html(str(tables[0]))[0]
content2 = pd.read_html(str(tables[1]))[0]
content3 = pd.read_html(str(tables[2]))[0]
content4 = pd.read_html(str(tables[3]))[0]

In [175]:
content = content1.append(content2.append(content3.append(content4,ignore_index=True),ignore_index=True),ignore_index=True)

In [176]:
content

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2],NaN
1,JANUARY,7.0,The Legend of La Llorona,Saban Films,Patricia Harris Seeley (director/screenplay); ...,[3],NaN
2,JANUARY,7.0,The Commando,Saban Films,Asif Akbar (director); Koji Steven Sakai (scre...,[4],NaN
3,JANUARY,14.0,Scream,Paramount Pictures / Spyglass Media Group,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[5],NaN
4,JANUARY,14.0,Hotel Transylvania: Transformania,Amazon Studios / Columbia Pictures / Sony Pict...,"Jennifer Kluska, Derek Drymon (directors); Amo...",[6],NaN
...,...,...,...,...,...,...,...
245,DECEMBER,21.0,I Wanna Dance with Somebody,TriStar Pictures / Black Label Media,Kasi Lemmons (director); Anthony McCarten (scr...,NaN,[54]
246,DECEMBER,21.0,Wildcat,Amazon Studios,"Melissa Lesh, Trevor Frost (directors)",NaN,[168]
247,DECEMBER,25.0,Babylon,Paramount Pictures,Damien Chazelle (director/screenplay); Brad Pi...,NaN,[208]
248,DECEMBER,NaN,NaN,NaN,NaN,NaN,NaN


In [177]:
content.drop('Ref.', axis = 1, inplace = True)

In [178]:
content.rename(columns = {'.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.':'Ref'}, inplace = True)

In [179]:
content.shape


(250, 6)

In [180]:
content.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref
0,JANUARY,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2]
1,JANUARY,7.0,The Legend of La Llorona,Saban Films,Patricia Harris Seeley (director/screenplay); ...,[3]
2,JANUARY,7.0,The Commando,Saban Films,Asif Akbar (director); Koji Steven Sakai (scre...,[4]
3,JANUARY,14.0,Scream,Paramount Pictures / Spyglass Media Group,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[5]
4,JANUARY,14.0,Hotel Transylvania: Transformania,Amazon Studios / Columbia Pictures / Sony Pict...,"Jennifer Kluska, Derek Drymon (directors); Amo...",[6]


In [182]:
content_2022 = content[['Title', 'Cast and crew']]

In [183]:
content_2022

,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo..."
...,...,...
245,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...
246,Wildcat,"Melissa Lesh, Trevor Frost (directors)"
247,Babylon,Damien Chazelle (director/screenplay); Brad Pi...
248,NaN,NaN


In [184]:
tmdb = TMDb()
tmdb.api_key = '2946819469ace2bf8ccde8b306d41888'

In [185]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def source_genre(title):
      genres = []
      result = tmdb_movie.search(title)
      if not result:
        return np.NaN
      else:
        movie_id = result[0].id
        response = re.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
        data_json = response.json()
        if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
        else:
            np.NaN

In [186]:
content_2022['genres'] = content['Title'].map(lambda x: source_genre(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [187]:
content_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Horror Thriller
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo...",Animation Family Fantasy Comedy Adventure
...,...,...,...
245,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama History
246,Wildcat,"Melissa Lesh, Trevor Frost (directors)",Drama Thriller War
247,Babylon,Damien Chazelle (director/screenplay); Brad Pi...,Action Adventure Science Fiction Thriller
248,NaN,NaN,Comedy


In [192]:
content_2022.drop(labels = 248 , axis = 0, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [193]:
content_2022.drop(labels = 249 , axis = 0, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [194]:
content_2022

,Title,Cast and crew,genres
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Horror Thriller
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo...",Animation Family Fantasy Comedy Adventure
...,...,...,...
243,Shazam! Fury of the Gods,"David F. Sandberg (director); Henry Gayden, Ch...",Comedy Action Fantasy
244,Puss in Boots: The Last Wish,"Joel Crawford (director); Antonio Banderas, Sa...",Animation Adventure Comedy Family Fantasy
245,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama History
246,Wildcat,"Melissa Lesh, Trevor Frost (directors)",Drama Thriller War


In [195]:
content_2022.isnull().sum()

Title            0
Cast and crew    0
genres           1
dtype: int64

In [196]:
def director(title):
  if " (director)" in title:
    return title.split(" (director)")[0]
  elif " (directors)" in title:
    return title.split(" (directors)")[0]
  else:
    return title.split(" (director/screenplay)")[0]

In [197]:
content_2022['director'] = content_2022['Cast and crew'].map(lambda x: director(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [198]:
def actor1(x):
   return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [199]:
content_2022['actor 1'] = content_2022['Cast and crew'].map(lambda x: actor1(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [200]:
def actor2(x):
   if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
   else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [201]:
content_2022['actor 2'] = content_2022['Cast and crew'].map(lambda x: actor2(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [202]:
def actor3(x):
   if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
   else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [203]:
content_2022['actor 3'] = content_2022['Cast and crew'].map(lambda x: actor3(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [204]:
content_2022

,Title,Cast and crew,genres,director,actor 1,actor 2,actor 3
0,The 355,Simon Kinberg (director/screenplay); Theresa R...,Action Thriller,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...,Horror Thriller,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director...",Horror Mystery Thriller,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo...",Animation Family Fantasy Comedy Adventure,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn
...,...,...,...,...,...,...,...
243,Shazam! Fury of the Gods,"David F. Sandberg (director); Henry Gayden, Ch...",Comedy Action Fantasy,David F. Sandberg,Zachary Levi,Asher Angel,Jack Dylan Grazer
244,Puss in Boots: The Last Wish,"Joel Crawford (director); Antonio Banderas, Sa...",Animation Adventure Comedy Family Fantasy,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén
245,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama History,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci
246,Wildcat,"Melissa Lesh, Trevor Frost (directors)",Drama Thriller War,"Melissa Lesh, Trevor Frost",Melissa Lesh,Trevor Frost (directors),NaN


In [205]:
content_2022.drop('Cast and crew', axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [206]:
content_2022.head()

,Title,genres,director,actor 1,actor 2,actor 3
0,The 355,Action Thriller,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz
1,The Legend of La Llorona,Horror Thriller,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo
2,The Commando,Action Crime Thriller,Asif Akbar,Mickey Rourke,Michael Jai White,NaN
3,Scream,Horror Mystery Thriller,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega
4,Hotel Transylvania: Transformania,Animation Family Fantasy Comedy Adventure,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn


In [207]:
content_2022.isnull().sum()

Title        0
genres       1
director     0
actor 1      0
actor 2      4
actor 3     19
dtype: int64

In [208]:
content_2022.rename(columns = {'genres':'Genre'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [210]:
content_2022 = content_2022.loc[:,['director','actor 1','actor 2','actor 3','Genre','Title']]

In [211]:
content_2022.head()

,director,actor 1,actor 2,actor 3,Genre,Title
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Horror Thriller,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,NaN,Action Crime Thriller,The Commando
3,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream
4,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Family Fantasy Comedy Adventure,Hotel Transylvania: Transformania


In [213]:
content_2022.isnull().sum()

director     0
actor 1      0
actor 2      4
actor 3     19
Genre        1
Title        0
dtype: int64

In [214]:
content_2022['actor 2'] = content_2022['actor 2'].replace(np.nan, 'unknown')
content_2022['actor 3'] = content_2022['actor 3'].replace(np.nan, 'unknown')
content_2022['Genre'] = content_2022['Genre'].replace(np.nan, 'unknown')

In [215]:
content_2022

,director,actor 1,actor 2,actor 3,Genre,Title
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,The 355
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Horror Thriller,The Legend of La Llorona
2,Asif Akbar,Mickey Rourke,Michael Jai White,unknown,Action Crime Thriller,The Commando
3,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream
4,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Family Fantasy Comedy Adventure,Hotel Transylvania: Transformania
...,...,...,...,...,...,...
243,David F. Sandberg,Zachary Levi,Asher Angel,Jack Dylan Grazer,Comedy Action Fantasy,Shazam! Fury of the Gods
244,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Animation Adventure Comedy Family Fantasy,Puss in Boots: The Last Wish
245,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama History,I Wanna Dance with Somebody
246,"Melissa Lesh, Trevor Frost",Melissa Lesh,Trevor Frost (directors),unknown,Drama Thriller War,Wildcat


In [216]:
content_2022['combo'] = content_2022['actor 1'] + content_2022['actor 2'] + ' ' + content_2022['actor 3'] + ' ' + content_2022['director'] + ' ' + content_2022['Genre']

In [217]:
content_2022

,director,actor 1,actor 2,actor 3,Genre,Title,combo
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,The 355,Jessica ChastainLupita Nyong'o Penélope Cruz S...
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Horror Thriller,The Legend of La Llorona,Autumn ReeserAntonio Cupo Danny Trejo Patricia...
2,Asif Akbar,Mickey Rourke,Michael Jai White,unknown,Action Crime Thriller,The Commando,Mickey RourkeMichael Jai White unknown Asif Ak...
3,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,Scream,Melissa BarreraMason Gooding Jenna Ortega Matt...
4,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Family Fantasy Comedy Adventure,Hotel Transylvania: Transformania,Andy SambergSelena Gomez Kathryn Hahn Jennifer...
...,...,...,...,...,...,...,...
243,David F. Sandberg,Zachary Levi,Asher Angel,Jack Dylan Grazer,Comedy Action Fantasy,Shazam! Fury of the Gods,Zachary LeviAsher Angel Jack Dylan Grazer Davi...
244,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Animation Adventure Comedy Family Fantasy,Puss in Boots: The Last Wish,Joel Crawford (director); Antonio BanderasSalm...
245,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama History,I Wanna Dance with Somebody,Naomi AckieAshton Sanders Stanley Tucci Kasi L...
246,"Melissa Lesh, Trevor Frost",Melissa Lesh,Trevor Frost (directors),unknown,Drama Thriller War,Wildcat,Melissa LeshTrevor Frost (directors) unknown M...


In [218]:
content_21 = pd.read_csv('/content/processed_21.csv')

In [219]:
content_21

,Unnamed: 0,director,actor 1,actor 2,actor 3,Genre,Title,combo
0,0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...,...
6504,6504,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery,The King's Man,Ralph FiennesGemma Arterton Rhys Ifans Matthew...
6505,6505,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,The Tragedy of Macbeth,Denzel WashingtonFrances McDormand Bertie Carv...
6506,6506,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,A Journal for Jordan,Michael B. JordanChanté Adams Jalon Christian ...
6507,6507,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,American Underdog,Zachary LeviAnna Paquin Dennis Quaid Erwin bro...


In [220]:
content_21.drop('Unnamed: 0', axis = 1, inplace = True)

In [221]:
content_21

,director,actor 1,actor 2,actor 3,Genre,Title,combo
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6504,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery,The King's Man,Ralph FiennesGemma Arterton Rhys Ifans Matthew...
6505,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,The Tragedy of Macbeth,Denzel WashingtonFrances McDormand Bertie Carv...
6506,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,A Journal for Jordan,Michael B. JordanChanté Adams Jalon Christian ...
6507,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,American Underdog,Zachary LeviAnna Paquin Dennis Quaid Erwin bro...


In [222]:
processed_22 = content_21.append(content_2022, ignore_index = True)

In [223]:
processed_22

,director,actor 1,actor 2,actor 3,Genre,Title,combo
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6752,David F. Sandberg,Zachary Levi,Asher Angel,Jack Dylan Grazer,Comedy Action Fantasy,Shazam! Fury of the Gods,Zachary LeviAsher Angel Jack Dylan Grazer Davi...
6753,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Animation Adventure Comedy Family Fantasy,Puss in Boots: The Last Wish,Joel Crawford (director); Antonio BanderasSalm...
6754,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama History,I Wanna Dance with Somebody,Naomi AckieAshton Sanders Stanley Tucci Kasi L...
6755,"Melissa Lesh, Trevor Frost",Melissa Lesh,Trevor Frost (directors),unknown,Drama Thriller War,Wildcat,Melissa LeshTrevor Frost (directors) unknown M...


In [224]:
processed_22.tail()

,director,actor 1,actor 2,actor 3,Genre,Title,combo
6752,David F. Sandberg,Zachary Levi,Asher Angel,Jack Dylan Grazer,Comedy Action Fantasy,Shazam! Fury of the Gods,Zachary LeviAsher Angel Jack Dylan Grazer Davi...
6753,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén,Animation Adventure Comedy Family Fantasy,Puss in Boots: The Last Wish,Joel Crawford (director); Antonio BanderasSalm...
6754,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci,Drama History,I Wanna Dance with Somebody,Naomi AckieAshton Sanders Stanley Tucci Kasi L...
6755,"Melissa Lesh, Trevor Frost",Melissa Lesh,Trevor Frost (directors),unknown,Drama Thriller War,Wildcat,Melissa LeshTrevor Frost (directors) unknown M...
6756,Damien Chazelle,Brad Pitt,Margot Robbie,Tobey Maguire,Action Adventure Science Fiction Thriller,Babylon,Brad PittMargot Robbie Tobey Maguire Damien Ch...


In [225]:
processed_22.to_csv('processed_22.csv')